In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr
from collections import Counter

# importamos as libs
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 
import numpy as np
import tensorflow as tf
sns.set_palette("Set2")

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
# calcula metricas de regressao
def score_regression_metrics(y_test, y_test_pred):

    RMSE = mean_squared_error(y_true=y_test, y_pred=y_test_pred, squared=False)
    MAE = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
    MAPE = mean_absolute_percentage_error(y_true=y_test, y_pred=y_test_pred)
    R2 = r2_score(y_true=y_test, y_pred=y_test_pred)

    scores = {
        "neg_root_mean_squared_error": RMSE,
        "neg_mean_absolute_error": MAE,
        "neg_mean_absolute_percentage_error": MAPE,
    }

    return scores

def show_scores (scores):

    print ("Scores obtidos:")

    print (f"RMSE: {scores['neg_root_mean_squared_error']}")
    print (f"MAE: {scores['neg_mean_absolute_error']}")
    print (f"MAPE: {scores['neg_mean_absolute_percentage_error']}")

In [3]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [4]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [32]:
def set_seeds():
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    os.environ['PYTHONHASHSEED'] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

In [5]:
df = pd.read_csv("input_and_output.csv")

In [33]:
SEED=47

TO_DROP = [
    "pib_pc",
    "idh_long",
    "nfsp",
    "estoque",
]

VARS = [
    'pib_pmc',
    'pib_pcpt',
    'populacao',
    'pib_cc',
    'ipca',
    'incc',
    'igp',
    'selic',
    'idh_renda',
    'idh_educacao',
    'desemprego'
]

batch_size = 32
epochs = 200
verbose = 1
timesteps = 3

In [7]:
# TO_DROP = [

# ]

# VARS = [
#     "pib_pc",
#     "idh_long",
#     "nfsp",
#     "estoque",
#     'pib_pmc',
#     'pib_pcpt',
#     'populacao',
#     'pib_cc',
#     'ipca',
#     'incc',
#     'igp',
#     'selic',
#     'idh_renda',
#     'idh_educacao',
#     'desemprego'
# ]

In [9]:
df = df.drop(TO_DROP, axis=1)

In [10]:
# for var in VARS:
#     df[var] = df[var].clip(upper=df[var].quantile(.75))
    

In [11]:
# for var in VARS:
#     df[var] = df[var].clip(upper=df[var].quantile(.75))
    

In [12]:
# df[VARS].hist(bins=25, figsize=(20, 20), layout=(-1, 5), edgecolor="black")
# plt.tight_layout()

In [51]:
x = df.drop(["data", "consumo"], axis=1).copy()

# processo de one-hot
x = pd.get_dummies(data=x, columns=["estados"], drop_first=True)

# aqui só precisamos do valor do consumo em si 
y = df[["consumo"]].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)

In [52]:
regr = LinearRegression()

regr.fit(x_train, y_train)

print(f"Score: {regr.score(x_test, y_test)}")

Score: 0.9040287172504055


In [53]:
print("================")
print("TRAIN")
print("================")
y_pred = regr.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)
print()
print("================")
print("TESTE")
print("================")
y_pred = regr.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 728354.1570515295
MAE: 459842.9989768059
MAPE: 0.5743427085647832

TESTE
Scores obtidos:
RMSE: 729794.0032279178
MAE: 464649.91215264314
MAPE: 0.5904931776252598


In [54]:
x = df.drop(["data", "estados", "consumo"], axis=1).copy()

# aqui só precisamos do valor do consumo em si 
y = df[["consumo"]].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)

In [55]:
regr = LinearRegression()

regr.fit(x_train, y_train)

print(f"Score: {regr.score(x_test, y_test)}")

Score: 0.844156189958258


In [56]:
print("================")
print("TRAIN")
print("================")
y_pred = regr.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)
print()
print("================")
print("TESTE")
print("================")
y_pred = regr.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 959893.0819658525
MAE: 589453.8664613751
MAPE: 1.1919070713848006

TESTE
Scores obtidos:
RMSE: 929981.7354214025
MAE: 522448.1925750584
MAPE: 0.6701396521781601


In [30]:
def get_model():
    model = tf.keras.Sequential()
    #model.add(tf.keras.layers.Dense(units=256, activation="relu")),
    model.add(tf.keras.layers.Dense(units=128, activation="relu")),
    model.add(tf.keras.layers.Dense(units=64, activation="relu")),
    model.add(tf.keras.layers.Dense(units=32, activation="relu")),
    model.add(tf.keras.layers.Dropout(rate=0.10))
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [31]:
x = df.drop(["estados", "data"], axis=1)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
set_seeds()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

Epoch 1/200
250/250 [==============================] - 3s 7ms/step - loss: 7916329893888.0000 - RMSE: 2813597.2500
Epoch 2/200
250/250 [==============================] - 1s 6ms/step - loss: 7606600990720.0000 - RMSE: 2758006.7500
Epoch 3/200
250/250 [==============================] - 1s 6ms/step - loss: 5944956157952.0000 - RMSE: 2438228.0000
Epoch 4/200
250/250 [==============================] - 1s 6ms/step - loss: 3021212221440.0000 - RMSE: 1738163.5000
Epoch 5/200
250/250 [==============================] - 1s 5ms/step - loss: 1360166518784.0000 - RMSE: 1166261.7500
Epoch 6/200
250/250 [==============================] - 2s 7ms/step - loss: 921011290112.0000 - RMSE: 959693.3125
Epoch 7/200
250/250 [==============================] - 1s 6ms/step - loss: 772019978240.0000 - RMSE: 878646.6875TA: 
Epoch 8/200
250/250 [==============================] - 2s 7ms/step - loss: 670445469696.0000 - RMSE: 818807.3750
Epoch 9/200
250/250 [==============================] - 1s 6ms/step - loss: 5429680

In [34]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 12576.403887420529
MAE: 9357.830837144036
MAPE: 0.02827587341015946
TESTE
Scores obtidos:
RMSE: 17529.11764329167
MAE: 11708.51924058067
MAPE: 0.0502694336291905


In [27]:
#x = df.drop(["estados", "data"], axis=1)
x = pd.get_dummies(data=df.drop(["data"], axis=1), columns=["estados"], drop_first=True)
y = df["consumo"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

Epoch 1/200
250/250 [==============================] - 3s 5ms/step - loss: 7895401365504.0000 - RMSE: 2809875.7500
Epoch 2/200
250/250 [==============================] - 2s 7ms/step - loss: 6905841844224.0000 - RMSE: 2627896.7500
Epoch 3/200
250/250 [==============================] - 2s 7ms/step - loss: 3050340876288.0000 - RMSE: 1746522.5000
Epoch 4/200
250/250 [==============================] - 1s 6ms/step - loss: 818758352896.0000 - RMSE: 904852.6875
Epoch 5/200
250/250 [==============================] - 2s 6ms/step - loss: 600752914432.0000 - RMSE: 775082.5000
Epoch 6/200
250/250 [==============================] - 1s 6ms/step - loss: 548182917120.0000 - RMSE: 740393.7500
Epoch 7/200
250/250 [==============================] - 1s 5ms/step - loss: 512521928704.0000 - RMSE: 715906.3750
Epoch 8/200
250/250 [==============================] - 2s 6ms/step - loss: 469646180352.0000 - RMSE: 685307.3750
Epoch 9/200
250/250 [==============================] - 1s 5ms/step - loss: 447186075648.00

In [29]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 70592.9345211642
MAE: 47825.52684364173
MAPE: 0.1773794592301615
TESTE
Scores obtidos:
RMSE: 74500.3454443094
MAE: 54028.48803054077
MAPE: 0.18284895261072728


In [20]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=32, activation="relu", return_sequences=True)),
    model.add(tf.keras.layers.GlobalAveragePooling1D()),
    model.add(tf.keras.layers.Dropout(rate=0.10))
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [21]:
x, y = split_sequences(df.drop(["estados", "data"], axis=1).values, timesteps)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler3DShape()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

Epoch 1/200
250/250 [==============================] - 10s 16ms/step - loss: 7925261139968.0000 - RMSE: 2815184.0000
Epoch 2/200
250/250 [==============================] - 3s 10ms/step - loss: 7904064700416.0000 - RMSE: 2811416.7500
Epoch 3/200
250/250 [==============================] - 2s 10ms/step - loss: 7741244964864.0000 - RMSE: 2782309.2500
Epoch 4/200
250/250 [==============================] - 3s 14ms/step - loss: 7281739563008.0000 - RMSE: 2698469.7500
Epoch 5/200
250/250 [==============================] - 3s 12ms/step - loss: 6568524382208.0000 - RMSE: 2562913.2500
Epoch 6/200
250/250 [==============================] - 3s 13ms/step - loss: 5942512451584.0000 - RMSE: 2437727.0000
Epoch 7/200
250/250 [==============================] - 3s 14ms/step - loss: 5577732259840.0000 - RMSE: 2361722.2500
Epoch 8/200
250/250 [==============================] - 3s 11ms/step - loss: 5354188963840.0000 - RMSE: 2313912.0000
Epoch 9/200
250/250 [==============================] - 3s 11ms/step - l

In [22]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 1643531.8119198007
MAE: 961708.9786198909
MAPE: 3.218955152249913
TESTE
Scores obtidos:
RMSE: 1856253.1239655064
MAE: 1025329.4853855793
MAPE: 3.2773748794606314


In [57]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=128, activation="relu", return_sequences=True)),
    model.add(tf.keras.layers.GlobalAveragePooling1D()),
    model.add(tf.keras.layers.Dropout(rate=0.10))
    model.add(tf.keras.layers.Dense(units=64, activation="relu")),
    model.add(tf.keras.layers.Dense(units=32, activation="relu")),
    model.add(tf.keras.layers.Dense(units=1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [58]:
x, y = split_sequences(df.drop(["estados", "data"], axis=1).values, timesteps)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler3DShape()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

Epoch 1/200
250/250 [==============================] - 11s 20ms/step - loss: 7178841227264.0000 - RMSE: 2679336.0000
Epoch 2/200
250/250 [==============================] - 4s 17ms/step - loss: 5093834358784.0000 - RMSE: 2256952.5000
Epoch 3/200
250/250 [==============================] - 4s 17ms/step - loss: 4868188667904.0000 - RMSE: 2206397.2500
Epoch 4/200
250/250 [==============================] - 4s 16ms/step - loss: 4804660690944.0000 - RMSE: 2191953.5000 1s - loss:
Epoch 5/200
250/250 [==============================] - 4s 17ms/step - loss: 4740293853184.0000 - RMSE: 2177221.5000
Epoch 6/200
250/250 [==============================] - 4s 18ms/step - loss: 4695315709952.0000 - RMSE: 2166867.7500
Epoch 7/200
250/250 [==============================] - 4s 18ms/step - loss: 4628081016832.0000 - RMSE: 2151297.5000
Epoch 8/200
250/250 [==============================] - 5s 20ms/step - loss: 4587365335040.0000 - RMSE: 2141813.5000
Epoch 9/200
250/250 [==============================] - 4s 17

In [59]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 443929.12511484534
MAE: 282512.7996313748
MAPE: 0.5220022297861832
TESTE
Scores obtidos:
RMSE: 977980.0045355747
MAE: 506618.1719305113
MAPE: 0.8634618847441115


In [60]:
x, y = split_sequences(df.drop(["estados", "data"], axis=1).values, 7)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
scaler = StandardScaler3DShape()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
model = get_model()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)

Epoch 1/200
250/250 [==============================] - 19s 40ms/step - loss: 5856031145984.0000 - RMSE: 2419923.7500
Epoch 2/200
250/250 [==============================] - 10s 38ms/step - loss: 4833391149056.0000 - RMSE: 2198497.5000
Epoch 3/200
250/250 [==============================] - 9s 36ms/step - loss: 4689267523584.0000 - RMSE: 2165471.7500
Epoch 4/200
250/250 [==============================] - 9s 36ms/step - loss: 4556190121984.0000 - RMSE: 2134523.5000
Epoch 5/200
250/250 [==============================] - 8s 34ms/step - loss: 4405464662016.0000 - RMSE: 2098920.0000
Epoch 6/200
250/250 [==============================] - 9s 37ms/step - loss: 4000290701312.0000 - RMSE: 2000072.6250
Epoch 7/200
250/250 [==============================] - 9s 38ms/step - loss: 3609022955520.0000 - RMSE: 1899742.8750
Epoch 8/200
250/250 [==============================] - 9s 36ms/step - loss: 3045112676352.0000 - RMSE: 1745025.1250
Epoch 9/200
250/250 [==============================] - 8s 32ms/step - 

In [61]:
print("================")
print("TRAIN")
print("================")
y_pred = model.predict(x_train)
scores = score_regression_metrics(y_train, y_pred)
show_scores(scores)

print("================")
print("TESTE")
print("================")
y_pred = model.predict(x_test)

scores = score_regression_metrics(y_test, y_pred)
show_scores(scores)

TRAIN
Scores obtidos:
RMSE: 423433.78527969075
MAE: 250094.0660791855
MAPE: 0.3648506572458502
TESTE
Scores obtidos:
RMSE: 1540075.196064732
MAE: 748130.5561810052
MAPE: 1.4345033264539517
